In [ ]:
# Used to test gibbs sampling function in python 
#Tested on school data

In [43]:
import pandas as pd
import numpy as np

In [44]:
school_data = pd.read_csv("https://www.scss.tcd.ie/~arwhite/Teaching/CS7DS3/school_compare_100.csv") 
print(len(school_data))
print(school_data['school'].nunique())
school_data.head()

1993
100


,school,mathscore
0,1,52.11
1,1,57.65
2,1,66.44
3,1,44.68
4,1,40.57


In [45]:
for i in ind:
    print(np.mean(y[ind == i]))
    break

50.81354838709678


In [50]:
def compare_m_gibbs(y, ind, mu0 = 50, gamma0 = 1/25,eta0 = 1/2, t0 = 50, a0 = 1/2, b0 = 50, maxiter = 1000):
    
    ### starting values
    m = ind.nunique()
    ybar = theta = [np.mean(y[ind == i]) for i in ind.unique()]
    tau_w = np.mean([1/np.var(y[ind == i]) for i in ind.unique()]) ##within group precision
    mu = np.mean(theta)
    tau_b = 1/np.var(theta) ##between group precision
    n_m = [len(y[ind == i]) for i in ind.unique()]
    an = a0 + sum(n_m)/2
    
    
    ### setup MCMC
    theta_mat = pd.DataFrame(columns = list(range(1,m+1)))
    mat_store = pd.DataFrame(columns = ["mu", "tau_w", "tau_b","theta_w", "theta_b"])
    
    for i in range(maxiter):
        
        # sample new values of the thetas
        theta = []
        for j in range(m): 
        
            taun = n_m[j]*tau_w + tau_b
            thetan = (ybar[j] * n_m[j] * tau_w + mu * tau_b) / taun
            theta.append(np.random.normal(thetan, np.sqrt(1/taun)))
        
        #sample new value of tau_w
        
        ss = 0
        for j in range(m):
            ss = ss + sum([ (x - theta[j])**2 for x in y[ind == j+1]])
        
        bn = b0 + ss/2
        tau_w = np.random.gamma(an, 1/bn)
        
        #sample a new value of mu
        gammam = m * tau_b + gamma0
        mum = (np.mean(theta) * m * tau_b + mu0 * gamma0) / gammam
        mu = np.random.normal(mum, np.sqrt(1/gammam))

        # sample a new value of tau_b
        etam = eta0 + m/2
        tm = t0 + sum([(t-mu)**2 for t in theta])/2
        tau_b = np.random.gamma(etam, 1/tm)

        #store results
        theta_mat.loc[i] = theta
        mat_store.loc[i] = [mu, tau_w, tau_b,1/np.sqrt(tau_w),1/np.sqrt(tau_b) ]
        
        if i%500 == 0: print("{}/{}".format(i,maxiter)) 
        
    return (theta_mat,mat_store)
        




0/1000
500/1000


In [ ]:
y = school_data['mathscore']
ind = school_data['school']
theta_mat,mat_store = compare_m_gibbs(y,ind)

,mu,tau_w,tau_b,theta_w,theta_b
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,48.124700,0.011788,0.041092,9.214436,4.986811
std,0.548840,0.000396,0.006987,0.155167,0.426671
min,46.114783,0.010499,0.023956,8.779654,3.788455
25%,47.746644,0.011526,0.036307,9.109848,4.690537
50%,48.122745,0.011791,0.040505,9.209082,4.968732
75%,48.505430,0.012050,0.045452,9.314383,5.248142
max,49.727231,0.012973,0.069675,9.759643,6.460910


In [28]:
def gibbs_difference(y, ind, mu0 = 50, tau0 = 1/625, del0 = 0, gamma0 = 1/625, a0 = 0.5, b0 = 50, maxiter = 5000):
    y1 = y[ind == 1]
    y2 = y[ind == 2]
  
    n1 = len(y1) 
    n2 = len(y2)
    
    #initial values
    mu = (y1.mean() + y2.mean()) / 2
    delta = (y1.mean() - y2.mean()) / 2
    
    df_samples = pd.DataFrame(columns=["mu", "del", "tau",'theta'])
    
    ##### Gibbs sampler
    an = a0 + (n1 + n2)/2
    
    for i in range(maxiter):
        
        ##update tau
        bn = b0 + 0.5 * (sum((y1 - mu - delta)**2) + sum((y2 - mu + delta)**2))
        tau = np.random.gamma(an, 1/bn)
        ##

        ##update mu
        taun =  tau0 + tau * (n1 + n2)
        mun = (tau0 * mu0 + tau * (sum(y1 - delta) + sum(y2 + delta))) / taun
        mu = np.random.normal(mun, np.sqrt(1/taun))  
        ##

        ##update delta
        gamman =  gamma0 + tau*(n1 + n2)
        deln = ( del0 * gamma0 + tau * (sum(y1 - mu) - sum(y2 - mu))) / gamman
        delta = np.random.normal(deln, np.sqrt(1/gamman))  
        
        df_samples.loc[i] = [mu, delta, tau,1/np.sqrt(tau)]
    
    return df_samples
    
    
y = school_data['score']
ind = school_data['school']
samples = gibbs_difference(y,ind)

In [29]:
samples.head()

,mu,del,tau,theta
0,48.690041,2.389765,0.009961,10.019427
1,48.688958,2.660109,0.011512,9.320293
2,48.415087,4.265700,0.009122,10.470285
3,46.340659,2.203715,0.007762,11.350193
4,47.336401,0.124564,0.009493,10.263506


In [31]:
samples.describe()

,mu,del,tau,theta
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,48.495764,2.293868,0.009515,10.384370
std,1.379955,1.349859,0.001751,0.973989
min,43.129034,-3.763227,0.004227,7.662888
25%,47.582905,1.404360,0.008284,9.685855
50%,48.499860,2.254377,0.009416,10.305214
75%,49.408878,3.194538,0.010659,10.986788
max,54.154133,6.834917,0.017030,15.381477
